In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models, layers
from fenpreprocessing import fen_to_array
from tensorflow.keras.callbacks import EarlyStopping

import datetime
%load_ext tensorboard

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Setting paramaters on early stopping
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose=1,
                          mode='min',
                          restore_best_weights=True)

log_dir = "logs/fit/baseline" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [4]:
# Memory management, likely not necessary, but used as a safety as per the documentation recommendations on using GPUS

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


1 Physical GPUs, 1 Logical GPUs


In [5]:
small_train = pd.read_csv('fens/converted_train_partial.csv')
small_val = pd.read_csv('fens/converted_val_partial.csv')

In [8]:
small_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338855 entries, 0 to 338854
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   Position  338855 non-null  object 
 1   Target    338855 non-null  float64
dtypes: float64(1), object(1)
memory usage: 5.2+ MB


In [23]:
np.fromstring(small_val.Position[0].replace('\n', '')[1: -1], dtype=np.float, sep=' ')

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0.

In [29]:
small_train.iloc[0, :]

Position    [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
Target                                                      0
Name: 0, dtype: object

In [6]:
from data_generation import fix_positions, position_generator

train_gen = position_generator(small_train)


In [7]:
next(train_gen)

TypeError: fix_positions() got an unexpected keyword argument 'axis'

In [11]:
baseline_model = models.Sequential()
baseline_model.add(layers.Dense(832, input_shape=(832,), activation='relu'))
baseline_model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
baseline_model.add(layers.MaxPooling2D(2))
baseline_model.add(layers.Flatten())
baseline_model.add(layers.Dense(64, activation='relu'))
baseline_model.add(layers.Dense(1, activation='sigmoid'))
baseline_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=['acc'])
baseline_model.summary()

# Fitting the model
baseline_history = baseline_model.fit(small_train,
                    validation_data=small_val,
                    steps_per_epoch=100,
                    epochs=30,
                    callbacks=[earlystop, tensorboard_callback]
                    )

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 832)               693056    
_________________________________________________________________
dense_6 (Dense)              (None, 64)                53312     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 746,433
Trainable params: 746,433
Non-trainable params: 0
_________________________________________________________________


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().